<a href="https://colab.research.google.com/github/xarvel/DataScience/blob/master/RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [46]:
import tensorflow as tf

import numpy as np
import os
import time

In [47]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

In [48]:
def text_stats(text):
  sample_size = 250
  print(f'Sample {sample_size} characters:')
  print('-' * 80)
  # Take a look at the first 250 characters in text
  print(text[:sample_size])
  print('-' * 80)
  # length of text is the number of characters in it
  print(f'Length of text: {len(text)} characters')
  # The unique characters in the file
  vocab = sorted(set(text))
  print(f'{len(vocab)} unique characters')

  return vocab

In [49]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
vocab = text_stats(text)

Sample 250 characters:
--------------------------------------------------------------------------------
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

--------------------------------------------------------------------------------
Length of text: 1115394 characters
65 unique characters


In [50]:
SEQUENCE_LENGTH = 100

# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

# The embedding dimension
EMBEDDING_DIMENTION = 256

# Number of RNN units
RNN_UNITS = 1024

In [51]:
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab),
    mask_token=None
)
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(),
    invert=True,
    mask_token=None
)

In [ ]:
# Length of the vocabulary in StringLookup Layer
VOCAB_SIZE = len(ids_from_chars.get_vocabulary())

In [52]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [53]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)
sequences = ids_dataset.batch(SEQUENCE_LENGTH + 1, drop_remainder=True)
dataset = sequences.map(split_input_target)

In [54]:
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target: b'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


In [55]:
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True).prefetch(tf.data.experimental.AUTOTUNE)

dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

In [57]:
class RNNModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(
        rnn_units,
        return_sequences=True,
        return_state=True
    )
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [58]:
model = RnnModel(
    vocab_size=VOCAB_SIZE,
    embedding_dim=EMBEDDING_DIMENTION,
    rnn_units=RNN_UNITS
)

In [59]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 66) # (batch_size, sequence_length, vocab_size)


In [60]:
model.summary()

Model: "my_model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     multiple                  16896     
                                                                 
 gru_1 (GRU)                 multiple                  3938304   
                                                                 
 dense_1 (Dense)             multiple                  67650     
                                                                 
Total params: 4022850 (15.35 MB)
Trainable params: 4022850 (15.35 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [61]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

In [62]:
sampled_indices

array([19, 57, 65, 53, 35, 45, 44,  9, 55,  0, 27, 40, 49,  3, 44, 18, 19,
        2, 33, 26, 64, 27,  7, 44, 30, 13, 34, 21, 59, 53, 21, 52,  6, 62,
       10, 11, 64, 35, 22, 43, 42, 32, 26, 57, 34, 50, 50,  2,  6,  3, 45,
       47, 64, 28, 60, 12, 50, 62, 26, 10, 46, 25, 64, 42,  9,  2, 33, 19,
       21, 42, 32,  8, 26, 62, 60, 57, 63, 56, 16, 38,  1, 59, 40, 26, 51,
       32, 59, 47, 45, 45, 58,  6, 22,  3, 31, 12, 55, 34, 64, 11])

In [63]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b"der ta'en for you;\nWith all swift speed you must away to France.\n\nKING RICHARD II:\nNorthumberland, t"

Next Char Predictions:
 b"FrznVfe.p[UNK]Naj!eEF TMyN,eQ?UHtnHm'w3:yVIdcSMrUkk '!fhyOu;kwM3gLyc. TFHcS-MwurxqCY\ntaMlSthffs'I!R;pUy:"


In [64]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [65]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (64, 100, 66)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(4.1912036, shape=(), dtype=float32)


In [66]:
tf.exp(example_batch_mean_loss).numpy()

66.1023

In [67]:
model.compile(optimizer='adam', loss=loss)

In [68]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [69]:
EPOCHS = 20

In [70]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/20
172/172 [==============================] - 14s 54ms/step - loss: 2.7174
Epoch 2/20
172/172 [==============================] - 11s 54ms/step - loss: 1.9869
Epoch 3/20
172/172 [==============================] - 11s 55ms/step - loss: 1.7057
Epoch 4/20
172/172 [==============================] - 11s 56ms/step - loss: 1.5456
Epoch 5/20
172/172 [==============================] - 12s 60ms/step - loss: 1.4477
Epoch 6/20
172/172 [==============================] - 14s 56ms/step - loss: 1.3807
Epoch 7/20
172/172 [==============================] - 11s 58ms/step - loss: 1.3276
Epoch 8/20
172/172 [==============================] - 12s 56ms/step - loss: 1.2819
Epoch 9/20
172/172 [==============================] - 13s 58ms/step - loss: 1.2420
Epoch 10/20
172/172 [==============================] - 12s 59ms/step - loss: 1.2024
Epoch 11/20
172/172 [==============================] - 13s 59ms/step - loss: 1.1628
Epoch 12/20
172/172 [==============================] - 13s 61ms/step - loss: 1.1213
E

In [71]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [72]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [73]:
def predict_text(start_text, size):
  start = time.time()
  states = None
  next_char = tf.constant([start_text])
  result = [next_char]

  for n in range(size):
    next_char, states = one_step_model.generate_one_step(next_char, states=states)
    result.append(next_char)

  result = tf.strings.join(result)
  end = time.time()
  print(result[0].numpy().decode('utf-8'), '\n\n' + '_' * 80)
  print('\nRun time:', end - start)

In [74]:
predict_text('ROMEO:', 1000)

ROMEO:

HORTENSI:
What, have thou made?

GRUMIO:
Let me see: yet you marge me the counterfeit
That you be at askance.

MENENIUS:
Why, sir, I am a Roman, and still your hearts,
That they must upper, wife, if he lives like in ill
thing to vice the time begin once perform 'gainst to
Be kept to be those at your husband.

PRISPERO:
If it be so, sir, I say!
Be there a madman to this pleasure, but, you are
the first waying angel on the people.

First Citizen:
To lose it is; for by his fellow spend;
Which I'll be you, sir, unkindness people.

Capston:
Not after me, I bare action.

GRUMIO:
Why Ispead, withal marry it without asking
How many war calls not thy followers fight,
We'll hear it, and we will still we may they
behind the gidst as much before us.

MENENIUS:
One of his head, thou must be past.

CORIOLANUS:
It is before the Duke of York is ruship;
Nay, head you do, and to thyself are present
doth buing the peace. I cannot deny:
Be up-deep-taintenance.

MISTRESS OVERDONE:
Well said, mascen

In [75]:
tf.saved_model.save(one_step_model, 'one_step')
one_step_reloaded = tf.saved_model.load('one_step')

In [76]:
class CustomTraining(RNNModel):
  @tf.function
  def train_step(self, inputs):
      inputs, labels = inputs
      with tf.GradientTape() as tape:
          predictions = self(inputs, training=True)
          loss = self.loss(labels, predictions)
      grads = tape.gradient(loss, model.trainable_variables)
      self.optimizer.apply_gradients(zip(grads, model.trainable_variables))

      return {'loss': loss}

In [77]:
model = CustomTraining(
    vocab_size=VOCAB_SIZE,
    embedding_dim=EMBEDDING_DIMENTION,
    rnn_units=RNN_UNITS
)

In [78]:
model.compile(optimizer = tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))

In [79]:
model.fit(dataset, epochs=1)

172/172 [==============================] - 15s 61ms/step - loss: 2.7316


In [80]:
EPOCHS = 10

mean = tf.metrics.Mean()

for epoch in range(EPOCHS):
    start = time.time()

    mean.reset_states()
    for (batch_n, (inp, target)) in enumerate(dataset):
        logs = model.train_step([inp, target])
        mean.update_state(logs['loss'])

        if batch_n % 50 == 0:
            template = f"Epoch {epoch+1} Batch {batch_n} Loss {logs['loss']:.4f}"
            print(template)

    # saving (checkpoint) the model every 5 epochs
    if (epoch + 1) % 5 == 0:
        model.save_weights(checkpoint_prefix.format(epoch=epoch))

    print()
    print(f'Epoch {epoch+1} Loss: {mean.result().numpy():.4f}')
    print(f'Time taken for 1 epoch {time.time() - start:.2f} sec')
    print("_"*80)

model.save_weights(checkpoint_prefix.format(epoch=epoch))

Epoch 1 Batch 0 Loss 2.2386
Epoch 1 Batch 50 Loss 2.0880
Epoch 1 Batch 100 Loss 1.9554
Epoch 1 Batch 150 Loss 1.8521

Epoch 1 Loss: 2.0028
Time taken for 1 epoch 13.48 sec
________________________________________________________________________________
Epoch 2 Batch 0 Loss 1.7988
Epoch 2 Batch 50 Loss 1.7886
Epoch 2 Batch 100 Loss 1.7438
Epoch 2 Batch 150 Loss 1.6415

Epoch 2 Loss: 1.7254
Time taken for 1 epoch 12.01 sec
________________________________________________________________________________
Epoch 3 Batch 0 Loss 1.6416
Epoch 3 Batch 50 Loss 1.5681
Epoch 3 Batch 100 Loss 1.5425
Epoch 3 Batch 150 Loss 1.5271

Epoch 3 Loss: 1.5615
Time taken for 1 epoch 11.86 sec
________________________________________________________________________________
Epoch 4 Batch 0 Loss 1.4879
Epoch 4 Batch 50 Loss 1.4781
Epoch 4 Batch 100 Loss 1.4505
Epoch 4 Batch 150 Loss 1.4478

Epoch 4 Loss: 1.4590
Time taken for 1 epoch 11.57 sec
_____________________________________________________________________